# Advanced Portfolio Optimization using cvxpy

## Install cvxpy and other libraries

In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt

    98% |███████████████████████████████▋| 870kB 11.0MB/s eta 0:00:01    54% |█████████████████▌              | 481kB 8.4MB/s eta 0:00:01    100% |████████████████████████████████| 880kB 8.2MB/s 
    100% |████████████████████████████████| 12.2MB 1.2MB/s eta 0:00:01  1% |▍                               | 153kB 13.4MB/s eta 0:00:01    22% |███████▏                        | 2.7MB 14.4MB/s eta 0:00:01    27% |████████▊                       | 3.3MB 11.1MB/s eta 0:00:01    38% |████████████▍                   | 4.7MB 28.4MB/s eta 0:00:01    53% |█████████████████▎              | 6.6MB 8.9MB/s eta 0:00:01    58% |██████████████████▉             | 7.1MB 10.2MB/s eta 0:00:01    68% |█████████████████████▉          | 8.3MB 7.7MB/s eta 0:00:01    78% |█████████████████████████       | 9.5MB 12.8MB/s eta 0:00:01    98% |███████████████████████████████▌| 12.0MB 27.7MB/s eta 0:00:01
    100% |████████████████████████████████| 26.2MB 591kB/s ta 0:00:011  0% |▏                               | 122kB 

## Imports

In [5]:
import cvxpy as cvx
import numpy as np
import quiz_tests_advanced

## What's our objective?
http://www.cvxpy.org/

Let's see how we can use optimization to meet a more advanced objective.  We want to both minimize the portfolio variance and also want to closely track a market cap weighted index.  In other words, we're trying to minimize the distance between the weights of our portfolio and the weights of the index.

$Minimize \left [ \sigma^2_p + \lambda \sqrt{\sum_{1}^{m}(weight_i - indexWeight_i)^2} \right  ]$ where $m$ is the number of stocks in the portfolio, and $\lambda$ is a scaling factor that you can choose.


## Hints

### x vector
To create a vector of M variables $\mathbf{x} = \begin{bmatrix}
x_1 &...& x_M
\end{bmatrix}
$
we can use `cvx.Variable(m)`

### covariance matrix
If we have $m$ stock series, the covariance matrix is an $m \times m$ matrix containing the covariance between each pair of stocks.  We can use [numpy.cov](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cov.html) to get the covariance.  We give it a 2D array in which each row is a stock series, and each column is an observation at the same period of time.

The covariance matrix $\mathbf{P} = 
\begin{bmatrix}
\sigma^2_{1,1} & ... & \sigma^2_{1,m} \\ 
... & ... & ...\\
\sigma_{m,1} & ... & \sigma^2_{m,m}  \\
\end{bmatrix}$

### portfolio variance
We can write the portfolio variance $\sigma^2_p = \mathbf{x^T} \mathbf{P} \mathbf{x}$

Recall that the $\mathbf{x^T} \mathbf{P} \mathbf{x}$ is called the quadratic form.
We can use the cvxpy function `quad_form(x,P)` to get the quadratic form.

### Distance from index weights
We want portfolio weights that track the index closely.  So we want to minimize the distance between them.
Recall from the Pythagorean theorem that you can get the distance between two points in an x,y plane by adding the square of the x and y distances and taking the square root.  Extending this to any number of dimensions is called the L2 norm.  So: $\sqrt{\sum_{1}^{n}(weight_i - indexWeight_i)^2}$  Can also be written as $\left \| \mathbf{x} - \mathbf{index} \right \|_2$.  There's a cvxpy function called [norm()](https://www.cvxpy.org/api_reference/cvxpy.atoms.other_atoms.html#norm)
`norm(x, p=2, axis=None)`.  The default is already set to find an L2 norm, so you would pass in one argument, which is the difference between your portfolio weights and the index weights.

### objective function
We want to minimize both the portfolio variance and the distance of the portfolio weights from the index weights.
We also want to choose a `scale` constant, which is $\lambda$ in the expression. This lets us choose how much priority we give to minimizing the difference from the index, relative to minimizing the variance of the portfolio.  If you choose a higher value for `scale` ($\lambda$), do you think this gives more priority to minimizing the difference, or minimizing the variance?

We can find the objective function using cvxpy `objective = cvx.Minimize()`.  Can you guess what to pass into this function?

### constraints
We can also define our constraints in a list.  For example, you'd want the weights to sum to one. So $\sum_{1}^{n}x = 1$.  You may also need to go long only, which means no shorting, so no negative weights.  So $x_i >0 $ for all $i$. you could save a variable as `[x >= 0, sum(x) == 1]`, where x was created using `cvx.Variable()`.

### optimization
So now that we have our objective function and constraints, we can solve for the values of $\mathbf{x}$.
cvxpy has the constructor `Problem(objective, constraints)`, which returns a `Problem` object.

The `Problem` object has a function solve(), which returns the minimum of the solution.  In this case, this is the minimum variance of the portfolio.

It also updates the vector $\mathbf{x}$.

We can check out the values of $x_A$ and $x_B$ that gave the minimum portfolio variance by using `x.value`

## Quiz

In [24]:
import cvxpy as cvx
import numpy as np

def optimize_portfolio(returns, index_weights, scale=.00001):
    """
    Create a function that takes the return series of a set of stocks, the index weights,
    and scaling factor. The function will minimize a combination of the portfolio variance
    and the distance of its weights from the index weights.  
    The optimization will be constrained to be long only, and the weights should sum to one.
    
    Parameters
    ----------
    returns : numpy.ndarray
        2D array containing stock return series in each row.
        
    index_weights : numpy.ndarray
        1D numpy array containing weights of the index.
        
    scale : float
        The scaling factor applied to the distance between portfolio and index weights
        
    Returns
    -------
    x : np.ndarray
        A numpy ndarray containing the weights of the stocks in the optimized portfolio
    """
    # TODO: Use cvxpy to determine the weights on the assets
    # that minimizes the combination of portfolio variance and distance from index weights
    
    print(returns)
    print(index_weights)
    
    # number of stocks m is number of rows of returns, and also number of index weights
    m = len(returns)
    

    #covariance matrix of returns
    # returned a 3 x 3 matrix for the 3 stocks
    cov = np.cov(returns)
    print(cov)
    
    # x variables (to be found with optimization)
    x = cvx.Variable(m)
    
    #portfolio variance, in quadratic form
    portfolio_variance = cvx.quad_form(x, cov)
    
    # euclidean distance (L2 norm) between portfolio and index weights
    distance_to_index = cvx.norm(x - index_weights)
    
    #objective function
    objective = cvx.Minimize(portfolio_variance + scale * distance_to_index)
    
    #constraints
    constraints = [x >= 0, sum(x) == 1]

    #use cvxpy to solve the objective
    problem = cvx.Problem(objective, constraints)
    result = problem.solve()
    min_value = problem.value
    print(min_value)
    print(x.value)
    x_values = x.value
    
    #retrieve the weights of the optimized portfolio
    #x_values
    
    return x_values

quiz_tests_advanced.test_optimize_portfolio(optimize_portfolio)

[[ 4.27432596e-01  5.79924699e-01 -5.04554530e-02  3.43589540e-01
   4.34303904e-01 -1.72289299e-02 -1.34473036e-02 -1.96783239e-01
  -1.71525506e-01 -4.03971944e-02  4.05439585e-01  5.50383273e-01
   3.51296435e-01  1.00935981e-01 -1.14530737e-01  6.45129238e-02
  -7.48591748e-02  3.15190034e-02  4.37228462e-02  1.35271708e-01
  -2.64555928e-01 -1.57283956e-01  5.18486863e-01 -2.73646929e-01
   4.50407198e-01 -3.89538370e-01 -4.41154201e-01  4.56103060e-02
   2.63475766e-01  1.64313657e-01 -1.29621522e-01  1.16263088e-01
   2.17561678e-01 -7.66092469e-02  2.26332067e-01  1.81448552e-01
  -1.56109284e-01  1.76243014e-02 -1.93220218e-02 -9.81702075e-02
   1.13705131e-01  2.55879845e-02  1.66037987e-01 -2.92391652e-01
  -4.11125673e-01 -4.99158672e-02 -1.17227701e-01 -1.77906713e-01
  -1.23995262e-01  3.43832651e-01 -3.83771537e-01  1.66972528e-01
   2.22169917e-01  2.41252941e-01  6.58248070e-02 -4.39844685e-01
  -3.17577191e-02 -4.00213697e-01  2.60458775e-01  3.45356405e-01
  -2.73541

In [25]:
"""Test with a 3 simulated stock return series"""
days_per_year = 252
years = 3
total_days = days_per_year * years

return_market = np.random.normal(loc=0.05, scale=0.3, size=days_per_year)
return_1 = np.random.uniform(low=-0.000001, high=.000001, size=days_per_year) + return_market
return_2 = np.random.uniform(low=-0.000001, high=.000001, size=days_per_year) + return_market
return_3 = np.random.uniform(low=-0.000001, high=.000001, size=days_per_year) + return_market
returns = np.array([return_1, return_2, return_3])

"""simulate index weights"""
index_weights = np.array([0.9,0.15,0.05])

"""try out your optimization function"""
x_values = optimize_portfolio(returns, index_weights, scale=.00001)

print(f"The optimized weights are {x_values}, which sum to {sum(x_values):.2f}")

[[ 0.7306941   0.19068053 -0.11297922 -0.36145563 -0.19986157 -0.39504408
   0.0011912   0.37379675  0.24123716 -0.02835702  0.32698661 -0.35843088
  -0.05442584  0.3268864   0.6643257   0.81046511  0.25504238  0.21465817
   0.04932978 -0.45882749  0.43971791 -0.00470218  0.09626867 -0.08698215
   0.39724136  0.41362635 -0.04890825  0.55696078  0.29247587  0.21852811
  -0.04465305  0.18808198 -0.36816974 -0.20018353 -0.65695625 -0.12213934
  -0.28038916  0.37297671  0.02558662 -0.40629444 -0.22147784  0.27209602
   0.13009112  0.26188472 -0.08155174 -0.11602817 -0.00124878  0.19293071
  -0.09709479  1.05503761 -0.21263377 -0.13875147 -0.5845375  -0.04953339
  -0.45520285  0.13269968  0.62660825 -0.0395277   0.8233402   0.30142994
  -0.38242395 -0.09737017 -0.11132829 -0.16636392 -0.24494674  0.63224183
  -0.27085747  0.07614484  0.21298358  0.57512768 -0.58151385 -0.2183161
   0.24741489  0.08247961  0.13978384  0.06189696  0.4778046  -0.12896906
  -0.17018842  0.11092666 -0.45781512 -

If you're feeling stuck, you can check out the solution [here](m3l4_cvxpy_advanced_solution.ipynb)